In [5]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [7]:
df = pd.read_csv("train.csv")

# Define a mapping dictionary for the classes
class_mapping = {
    'face_with_mask': 'face_with_mask',
    'mask_colorful': 'face_with_mask',
    'face_with_mask_incorrect': 'face_with_mask',
    'mask_surgical': 'face_with_mask',
    'face_other_covering': 'face_with_mask',
    'scarf_bandana': 'face_with_mask',
    
    'face_no_mask': 'face_no_mask',
    'eyeglasses': 'face_no_mask',
    'helmet': 'face_no_mask',
    'face_shield': 'face_no_mask',
    'sunglasses': 'face_no_mask',
    'hood': 'face_no_mask',
    'hat': 'face_no_mask',
    'goggles': 'face_no_mask',
    'hair_net': 'face_no_mask',
    'hijab_niqab': 'face_no_mask',
    'other': 'face_no_mask',
    'gas_mask': 'face_no_mask',
    'balaclava_ski_mask': 'face_no_mask',
    'turban': 'face_no_mask'
}



In [8]:
# Apply the mapping to the 'classname' column
df['classname'] = df['classname'].map(class_mapping)

# Filter only 'face_with_mask' and 'face_no_mask' classes
df = df[df['classname'].isin(['face_with_mask', 'face_no_mask'])]

# Preprocess the images and labels
images = []
labels = []
image_size = 128  # Resize images to 128x128

# Load and process each image
for index, row in df.iterrows():
    img_path = os.path.join("C:/Users/Hardik Nikam/Downloads/Dataset/Dataset/archive/Medical mask/Medical mask/Medical Mask/images", row['name'])
    img = cv2.imread(img_path)
    
    if img is not None:
        # Resize the image
        img = cv2.resize(img, (image_size, image_size))
        img = img.astype('float32') / 255.0
        images.append(img)
        labels.append(row['classname'])

In [9]:
# Convert images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode the labels ('face_with_mask' -> 0, 'face_no_mask' -> 1)
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)  # Convert to one-hot encoding

# Shuffle the data
images, labels = shuffle(images, labels, random_state=42)

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))  # Output layer for 2 classes

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(images, labels, epochs=10, batch_size=32)

# Save the trained model
# model.save("face_mask_detection_model.h5")

C:\Users\Hardik Nikam\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 150s 222ms/step - accuracy: 0.6571 - loss: 0.6555
Epoch 2/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 105s 212ms/step - accuracy: 0.6706 - loss: 0.6373
Epoch 3/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 211ms/step - accuracy: 0.6655 - loss: 0.6387
Epoch 4/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 103s 212ms/step - accuracy: 0.6674 - loss: 0.6352
Epoch 5/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 211ms/step - accuracy: 0.6597 - loss: 0.6305
Epoch 6/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 212ms/step - accuracy: 0.6699 - loss: 0.6166
Epoch 7/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 103s 213ms/step - accuracy: 0.6745 - loss: 0.6037
Epoch 8/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 212ms/step - accuracy: 0.6858 - loss: 0.5880
Epoch 9/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 211ms/step - accuracy: 0.6937 - loss: 0.5759
Epoch 10/10
482/482 ━━━━━━━━━━━━━━━━━━━━ 102s 212ms/step - accuracy: 0.7001 - loss: 0.5646


In [10]:
model.save("face_mask_detection_model_2.h5")